In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
# noinspection PyUnresolvedReferences
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def reformat(x_data):
    """
    Reformats the data to the format acceptable for convolutional layers
    :param x_data: input array
    :return: reshaped input and labels
    """
    img_size = int(np.sqrt(x_data.shape[-1]))
    return x_data.reshape((-1, img_size, img_size, 1)).astype(np.float32)

def load_data():
    """
    Function to (download and) load the MNIST data
    """
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    return (
        reformat(mnist.train.images),
        mnist.train.labels,
        reformat(mnist.validation.images),
        mnist.validation.labels,
    )


def randomize(x_data, y_data):
    """ Randomizes the order of data samples and their corresponding labels"""
    permutation = np.random.permutation(y_data.shape[0])
    return x_data[permutation, :, :, :], y_data[permutation]

In [3]:
IMAGE_HEIGHT, IMAGE_WIDTH = 28, 28

In [4]:
input_layer = tf.placeholder(tf.float32, shape=[None, IMAGE_HEIGHT, IMAGE_WIDTH, 1], name='X')

In [5]:
conv_layer_0_filter_size = 5
conv_layer_0_stride = 1
conv_layer_0_filters_number = 16  # what does this variable means ? Should be 23² (28 - 5) ^ 2

conv_layer_0_weights = tf.get_variable(
    'conv_layer_0_weights',
    dtype=tf.float32,
    shape=[
       conv_layer_0_filter_size,
       conv_layer_0_filter_size,
       input_layer.get_shape().as_list()[-1],
       conv_layer_0_filters_number
    ],
    initializer=tf.truncated_normal_initializer(stddev=0.01)
)

conv_layer_0_biases = tf.get_variable(
    "conv_layer_0_biases",
    dtype=tf.float32,
    initializer=tf.constant(0.0, shape=[conv_layer_0_filters_number], dtype=tf.float32)
)

conv_layer_0 = tf.nn.conv2d(
    input_layer,
    conv_layer_0_weights,
    strides=[1, conv_layer_0_stride, conv_layer_0_stride, 1],
    padding="SAME"
)

conv_layer_0 += conv_layer_0_biases
conv_layer_0 = tf.nn.relu(conv_layer_0)

In [6]:
pool_layer_0_size = 2
pool_layer_0 = tf.nn.max_pool(
    conv_layer_0,
    ksize=[1, pool_layer_0_size, pool_layer_0_size, 1],  # Why 4 dimensions ?
    strides=[1, pool_layer_0_size, pool_layer_0_size, 1],
    padding="SAME",
    name="pool_layer_0"
)

In [7]:
conv_layer_1_filter_size = 5
conv_layer_1_stride = 1
conv_layer_1_filters_number = 32  # what does this variable means ? Should be 23² (28 - 5) ^ 2

conv_layer_1_weights = tf.get_variable(
    'conv_layer_1_weights',
    dtype=tf.float32,
    shape=[
       conv_layer_1_filter_size,
       conv_layer_1_filter_size,
       pool_layer_0.get_shape().as_list()[-1],
       conv_layer_1_filters_number
    ],
    initializer=tf.truncated_normal_initializer(stddev=0.01)
)

conv_layer_1_biases = tf.get_variable(
    "conv_layer_1_biases",
    dtype=tf.float32,
    initializer=tf.constant(0.0, shape=[conv_layer_1_filters_number], dtype=tf.float32)
)

conv_layer_1 = tf.nn.conv2d(
    pool_layer_0,
    conv_layer_1_weights,
    strides=[1, conv_layer_1_stride, conv_layer_1_stride, 1],
    padding="SAME"
)

conv_layer_1 += conv_layer_1_biases
conv_layer_1 = tf.nn.relu(conv_layer_1)

In [8]:
pool_layer_1_size = 2
pool_layer_1 = tf.nn.max_pool(
    conv_layer_1,
    ksize=[1, pool_layer_1_size, pool_layer_1_size, 1],  # Why 4 dimensions ?
    strides=[1, pool_layer_1_size, pool_layer_1_size, 1],
    padding="SAME",
    name="pool_layer_1"
)

In [9]:
flat_layer = tf.reshape(
    tensor=pool_layer_1,
    shape=[-1, pool_layer_1.shape[1:4].num_elements()]
)

In [10]:
fc_layer_0_size = 128
fc_layer_0_weights = tf.get_variable(
    name='fc_layer_0_weights',
    dtype=tf.float32,
    shape=[flat_layer.shape[1], fc_layer_0_size],
    initializer=tf.truncated_normal_initializer(stddev=0.01),
)
fc_layer_0_biases = tf.get_variable(
    name='fc_layer_0_biases',
    dtype=tf.float32,
    initializer=tf.constant(0.0, shape=[fc_layer_0_size], dtype=tf.float32),
)

fc_layer_0 = tf.matmul(flat_layer, fc_layer_0_weights)
fc_layer_0 = tf.add(fc_layer_0, fc_layer_0_biases)
fc_layer_0 = tf.nn.relu(fc_layer_0)

In [11]:
CLASSES_NUMBER = 10
output_layer_weights = tf.get_variable(
    name='output_layer_weights',
    dtype=tf.float32,
    shape=[fc_layer_0.shape[1], CLASSES_NUMBER],
    initializer=tf.truncated_normal_initializer(stddev=0.01),
)
output_layer_biases = tf.get_variable(
    name='output_layer_biases',
    dtype=tf.float32,
    initializer=tf.constant(0.0, shape=[CLASSES_NUMBER], dtype=tf.float32),
)

output_layer = tf.matmul(fc_layer_0, output_layer_weights)
output_layer = tf.add(output_layer, output_layer_biases)

In [12]:
print(input_layer.shape)
print(conv_layer_0.shape)
print(pool_layer_0.shape)
print(conv_layer_1.shape)
print(pool_layer_1.shape)
print(flat_layer.shape)
print(fc_layer_0.shape)
print(output_layer.shape)

(?, 28, 28, 1)
(?, 28, 28, 16)
(?, 14, 14, 16)
(?, 14, 14, 32)
(?, 7, 7, 32)
(?, 1568)
(?, 128)
(?, 10)


In [13]:
output_train = tf.placeholder(
    dtype=tf.float32,
    shape=[None, CLASSES_NUMBER],
    name='output_train'
)
loss = tf.reduce_mean(
    input_tensor=tf.nn.softmax_cross_entropy_with_logits(
        labels=output_train,
        logits=output_layer,
    ),
    name='loss',
)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001, name='Adam-optimizer').minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [14]:
accuracy = tf.reduce_mean(
    input_tensor=tf.cast(
        x=tf.equal(
            x=tf.argmax(output_layer, 1),
            y=tf.argmax(output_train, 1),
            name='correct_prediction'
        ),
        dtype=tf.float32,
    ),
    name='accuracy',
)

In [15]:
x_train, y_train, x_valid, y_valid = load_data()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [19]:
# noinspection PyShadowingNames
def print_accuracy(loss, accuracy, input_layer, output_train, x_valid, y_valid, epoch_id):
    validation_data_loss, validation_data_accuracy = session.run(
            fetches=[loss, accuracy],
            feed_dict={input_layer: x_valid, output_train: y_valid}
        )
    print(f"Epoch: {epoch_id}, validation loss: {validation_data_loss:.2f}, validation accuracy: {validation_data_accuracy:.01%}" )
    print("-" * 60)

In [20]:
epochs_number = 10
batch_size = 100

initializer = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(initializer)
    for epoch_id in range(epochs_number):
        x_train, y_train = randomize(x_train, y_train)
        if epoch_id == 0:
            print_accuracy(loss, accuracy, input_layer, output_train, x_train, y_train, epoch_id)
        for batch_id in range(batch_size):
            start = batch_id * batch_size
            end = (batch_id + 1) * batch_size
            x_batch = x_train[start:end]
            y_batch = y_train[start:end]
            session.run(
                fetches=optimizer,
                feed_dict={
                    input_layer: x_batch,
                    output_train: y_batch
                }
            )
        print_accuracy(loss, accuracy, input_layer, output_train, x_train, y_train, epoch_id)

Epoch: 0, validation loss: 2.30, validation accuracy: 6.7%
------------------------------------------------------------
Epoch: 0, validation loss: 0.49, validation accuracy: 85.1%
------------------------------------------------------------
Epoch: 1, validation loss: 0.29, validation accuracy: 91.3%
------------------------------------------------------------
Epoch: 2, validation loss: 0.21, validation accuracy: 93.4%
------------------------------------------------------------
Epoch: 3, validation loss: 0.16, validation accuracy: 95.0%
------------------------------------------------------------
Epoch: 4, validation loss: 0.14, validation accuracy: 95.7%
------------------------------------------------------------
Epoch: 5, validation loss: 0.11, validation accuracy: 96.7%
------------------------------------------------------------
Epoch: 6, validation loss: 0.10, validation accuracy: 96.9%
------------------------------------------------------------
Epoch: 7, validation loss: 0.10, 